In [2]:
import cobra
import pandas as pd
import json
from custom_functions_scRBA import *

In [3]:
model = cobra.io.load_json_model('./cobraModel_zero.json')
model_raw = model.copy()

In [3]:
proteins = [met.id for met in model.metabolites if met.id[:4] == 'PRO-']
text = []
for p in proteins:
    rxn = cobra.Reaction('DMRXN-' + p)
    model.add_reaction(rxn)
    rxn.reaction = p + ' --> '
    
    model.objective = dict()
    model.reactions.get_by_id('DMRXN-' + p).objective_coefficient = 1
    fba = model.optimize()
    
    model.remove_reactions(['DMRXN-' + p])
    text.append([p, fba.status, str(fba.objective_value)])

In [4]:
text = ['\t'.join(i) for i in text]
with open('./test_results/proteins.txt', 'w') as f:
    f.write('\n'.join(text))

#### Test BIO-

In [5]:
bios = [met.id for met in model.metabolites if met.id[:4] == 'BIO-']
for b in bios:
    rxn = cobra.Reaction('DMRXN-' + b)
    model.add_reaction(rxn)
    rxn.reaction = b + ' --> '
    
    model.objective = dict()
    model.reactions.get_by_id('DMRXN-' + b).objective_coefficient = 1
    fba = model.optimize()
    
    model.remove_reactions(['DMRXN-' + b])
    print('\t'.join([b, fba.status, str(fba.objective_value)]))

BIO-modeledProtBiom_c	optimal	26.854300544194828
BIO-modeledProtBiom_m	optimal	26.854300544194725
BIO-rna_c	optimal	34.83750528287004
BIO-unmodeledProtBiom	optimal	26.854300544194768


In [10]:
p = 'PRO-preUnmodeledProtBiom'

rxn = cobra.Reaction('DMRXN-' + p)
model.add_reaction(rxn)
rxn.reaction = p + ' --> '

model.objective = dict()
model.reactions.get_by_id('DMRXN-' + p).objective_coefficient = 1
fba = model.optimize()

model.remove_reactions(['DMRXN-' + p])
print('\t'.join([p, fba.status, str(fba.objective_value)]))

PRO-preUnmodeledProtBiom	optimal	26.85430054419469


In [12]:
for i in fba.fluxes.index:
    try:
        _,rxn_base_id,_,_ = extract_details_from_rxnid(i)
    except:
        rxn_base_id = i
        
    if rxn_base_id.split('_')[-1] == 'm':
        print(i, fba.fluxes[i])

RXN-P5CD_m_FWD-YHR037W 0.0
RXN-P5CD_m_REV-YHR037W 0.0
RXN-HPPK2_m_FWD-YNL256W 0.0
RXN-HCITR_m_FWD-YDR234W 4.7310539819799935
RXN-HCITR_m_REV-YDR234W 0.0
RXN-MCITS_m_FWD-YPR001W 0.0
RXN-OMCDC_m_FWD-YHR208W 0.0
RXN-MTMOHT_m_FWD-YBR176W 0.0
RXN-4HBZF_m_FWD-DUMMYENZ_m 0.0
RXN-ALAS_m_FWD-YDR232W 0.0
RXN-KARA1i_m_FWD-YLR355C 16.031724520118427
RXN-ACOATA_m_FWD-YKL192C 0.0
RXN-ACOATA_m_REV-YKL192C 0.0
RXN-ACACT40ir_m_FWD-YPL028W 0.0
RXN-ACACT40ir_m_REV-YPL028W 0.0
RXN-ACCOAC_m_FWD-YMR207C 0.0
RXN-ACOAHi_m_FWD-YBL015W 0.0
RXN-ACGK_m_FWD-YER069W 10.344639741264116
RXN-ACONT2_m_FWD-YPR002W 0.0
RXN-ACONT2_m_REV-YPR002W 0.0
RXN-ACOTAi_m_FWD-YOL140W 10.344639741264116
RXN-COAHL_m_FWD-YPL148C 0.0
RXN-ADK1_m_FWD-YER170W 0.0
RXN-ADK3_m_FWD-YER170W 0.0
RXN-ADK3_m_REV-YER170W 0.0
RXN-ALCD2i2_m_FWD-YMR083W 0.0
RXN-ALCD2i2_m_FWD-YGL256W 0.0
RXN-ALCD22xi_m_FWD-YMR083W 0.0
RXN-ALCD22xi_m_FWD-YGL256W 0.0
RXN-ALCD25xi_m_FWD-YMR083W 0.0
RXN-ALCD25xi_m_FWD-YGL256W 0.0
RXN-ALDD2x_m_FWD-YOR374W 0.0
RXN-ALDD2y_m_F

In [14]:
model.reactions.get_by_id('RXN-PYRtps_m_FWD-MPC13').reaction

'MET-h_c + MET-pyr_c --> MET-h_m + MET-pyr_m'

In [7]:
df_rxns = pd.read_excel('./scRAM_network_frompy.xlsx')

In [16]:
df_rxns[df_rxns.reaction.str.contains('PRO-YEL039C')]

,id,type,symbolic_coeffs,reaction
4090,ENZSYN-CCP1CYC7,enzyme,NaN,PRO-YEL039C + PRO-YKR066C --> ENZ-CCP1CYC7


In [17]:
df_rxns[df_rxns.reaction.str.contains('PRO-YJR048W')]

,id,type,symbolic_coeffs,reaction
4091,ENZSYN-CCP1CYC1,enzyme,NaN,PRO-YJR048W + PRO-YKR066C --> ENZ-CCP1CYC1


In [18]:
df_rxns[df_rxns.reaction.str.contains('ENZ-CCP1CYC1')]

,id,type,symbolic_coeffs,reaction
3037,RXN-FECPOR_m_FWD-CCP1CYC1,metabolic,mu/keff,2 MET-focytC_m + MET-h2o2_m + 2 MET-h_m + mu/k...
4091,ENZSYN-CCP1CYC1,enzyme,NaN,PRO-YJR048W + PRO-YKR066C --> ENZ-CCP1CYC1


In [9]:
df_rxns[df_rxns.reaction.str.contains('PRO-preModeledProtBiom_m')]

,id,type,symbolic_coeffs,reaction
4204,PROSYN-Q0045,protein,mu*NAA/kribo,MET-fmettrna_c + MET-atp_c + 1069 MET-h2o_c + ...
4205,PROSYN-Q0080,protein,mu*NAA/kribo,MET-fmettrna_c + MET-atp_c + 97 MET-h2o_c + 3 ...
4206,PROSYN-Q0085,protein,mu*NAA/kribo,MET-fmettrna_c + MET-atp_c + 519 MET-h2o_c + 1...
4207,PROSYN-Q0105,protein,mu*NAA/kribo,MET-fmettrna_c + MET-atp_c + 771 MET-h2o_c + 1...
4208,PROSYN-Q0130,protein,mu*NAA/kribo,MET-fmettrna_c + MET-atp_c + 153 MET-h2o_c + 2...
...,...,...,...,...
5243,PROSYN-YDL119C,protein,mu*NAA/kribo,MET-fmettrna_c + MET-atp_c + 615 MET-h2o_c + 7...
5248,PROSYN-YBL080C,protein,mu*NAA/kribo,MET-fmettrna_c + MET-atp_c + 1083 MET-h2o_c + ...
5249,PROSYN-YGR102C,protein,mu*NAA/kribo,MET-fmettrna_c + MET-atp_c + 367 MET-h2o_c + 2...
5255,PROSYN-YFR049W,protein,mu*NAA/kribo,MET-fmettrna_c + MET-atp_c + 247 MET-h2o_c + 3...
